# Chapter 10. Monoids

##10.1. What is a Monoid?

In [ ]:
trait Monoid[A] {
    def op(a1:A, a2:A): A
    def zero: A
}

* A **_type_**
* \+ Binary operation _(**op**)_ over that type
* \+ Satisfying _**associativity**_ 
* \+ Identity element _(**zero**)_

```scala
- op(op(x,y), z) == op(x, op(y,z))  // x:A, y:A, z:A  // associativity
- op(x, zero) == op(zero, x) == x // for any x:A
```

### examples

```scala
// for string concatenation
"foo" + "bar"
(s + "") == ("" + s) == s
(r + s + t) == ((r+s)+t) == (r+(s+t))


// for integer addition
x + y
(x + 0) == (0 + x) == x
(x + y + z) == ((x+y)+z) == (x+(y+z))
```

## 10.2. Folding lists with monoids

In [ ]:
val stringMonoid = new Monoid[String] {
    def op(a1: String, a2: String) = a1 + a2
    val zero = ""
}

```scala
def foldRight[B](z: B)(f: (A,B) => B): B
def foldLeft[B](z: B)(f: (B,A) => B): B
```
#### _what if A==B ?_
```scala
def foldRight[A](z: A)(f: (A,A) => A): A
def foldLeft[A](z: A)(f: (A,A) => A): A
```

In [ ]:
val words = List("Hic", "Est", "Index")

In [7]:
val s = words.foldRight(stringMonoid.zero)(stringMonoid.op)

HicEstIndex

In [8]:
val t = words.foldLeft(stringMonoid.zero)(stringMonoid.op)

HicEstIndex

____
![](figures/foldStringMonoid2.PNG)

### 조금 더 일반화된 함수로 바꿔보자...

In [ ]:
def concatenate[A](as: List[A], m: Monoid[A]): A = 
    as.foldLeft(m.zero)(m.op)

In [10]:
concatenate(words, stringMonoid)

HicEstIndex

### Monoid 인스턴스가 없는 리스트에도 일반적으로 적용할 수 있을까??
=> Monoid 인스턴스가 있는 타입으로 항상 변환할 수가 있다면... 다음과 같은 일반화 함수가 가능.(<font color="red">map() 쓰지 않음!</font>)

In [ ]:
def foldMap[A,B](as: List[A], m:Monoid[B])(f: A => B): B =
    as.foldLeft(m.zero)((b,a) => m.op(b, f(a)))

In [25]:
/* Integer Monoid not defined above... */
val x = List(1,2,3)

List(1, 2, 3)

In [22]:
val foldedX = foldMap(x, stringMonoid)(_.toString())

123

## 10.3. Associativity and parallelism

### 평범한 folding

```scala
op(a, op(b, op(c, d)))
```
![](figures/ordinaryFold.PNG)

###### => 중간에 쓰이지 않고 버려지는 결과가 많음..

### "balanced" folding

```scala
op(op(a, b), op(c, d))
```

#### => 이점: op(a,b)와 op(c,d)를 병렬처리시킬 수 있음.
###### "loremipsum" + "dolorsit"

## 10.4. Example: Parallel parsing

### Monoid homomorphisms(호모모르피즘. 준동형(準同形). 대수적 구조를 보존한다는 의미..)

```scala
/* 아래에서 length함수는 String => Int로의 함수인데, Monoid 구조를 보존해준다.(뭔말인겨..??) */
```

In [27]:

"lorem".length + "ipsum".length == ("lorem"+"ipsum").length

true

## 10.5. Foldable data structures

- 'ints'라는 어떤 구조체가 정수들의 집합체라고 가정하면.. 집합체 안의 정수들의 합을 다음과 같이 구할 수 있음.
```scala
ints.foldRight(0)(_ + _)
```
- 이때 이 'ints' 구조의 형태가 List인지 Vector인지 Stream인지는 상관하지 않고 쓸 수 있다.
- F[\_]라는 형태로 'type **constructor**'를 구성할 수 있다.
- 아래의 Foldable 같은 것을 **_higher-order type constructor_** 또는 **_higher-kinded type_**이라 함.

In [ ]:
trait Foldable[F[_]] {
    def foldRight[A,B](as: F[A])(z: B)(f: (A,B) => B): B
    def foldLeft[A,B](as: F[A])(z: B)(f: (B,A) => B): B
    def foldMap[A,B](as: F[A])(f: A => B)(mb: Monoid[B]): B
    def concatename[A](as: F[A])(m: Monoid[A]): A =
        foldLeft(as)(m.zero)(m.op)
}

## 10.6. Composing Monoids

- Monoid의 진정한 파워는 'compose'에서부터...
- type A,B가 Monoid면 튜플 (A,B)도 또한 Monoid임.

In [ ]:
def productMonoid[A,B](A: Monoid[A], B: Monoid[B]): Monoid[(A, B)] =
  new Monoid[(A, B)] {
    def op(x: (A, B), y: (A, B)) =
      (A.op(x._1, y._1), B.op(x._2, y._2))
    val zero = (A.zero, B.zero)
  }

### 10.6.1 Assembling more complex monoids

![](figures/mergingMap.PNG)
![](figures/mergingMap2.PNG)

### 10.6.2. Using composed monoids to fuse traversals

- 책의 범위를 넘어가므로 chapter note를 참고하자.. :-P

## 10.7. Summary